In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [28]:
# Параметры для запроса
base_url = "https://api.geckoterminal.com/api/v2/networks/eth/pools"
params = {
    "include": ["base_token", "quote_token", "dex"],
    "sort": "h24_volume_usd_desc"
}
all_data = []

In [3]:
for page in range(1, 11):
    params['page'] = page  # Установка параметра страницы
    response = requests.get(base_url, params=params)
    
    # Проверка успешности запроса
    if response.status_code == 200:
        data = response.json()
        
        # Извлечение нужных данных
        if 'data' in data:
            all_data.extend(data['data'])  # Добавляем данные в список
    else:
        print(f"Ошибка на странице {page}: {response.status_code}")

In [ ]:
df = pd.json_normalize(all_data)
df.to_csv('geckoterminal_pools.csv', index=False)

In [152]:
df = pd.read_csv('geckoterminal_pools.csv')
df.shape

(200, 48)

In [153]:
df['created_time'] = pd.to_datetime(df['attributes.pool_created_at'], format='%Y-%m-%dT%H:%M:%SZ')
df['id'] = df['id'].str.replace('^eth_', '', regex=True)

In [154]:
df['attributes.reserve_in_usd'] = pd.to_numeric(df['attributes.reserve_in_usd'], errors='coerce')
filtered_df = df[df['attributes.reserve_in_usd'] > 1000000]
filtered_df = filtered_df.sort_values(by=['attributes.reserve_in_usd', "created_time"], ascending=False)
print(f"Количество подходящих строк : {len(filtered_df)}")

Количество подходящих строк : 133


In [155]:
filtered_df['created_time'] = pd.to_datetime(filtered_df['created_time'])
current_date = datetime.now()
five_months_ago = current_date - relativedelta(months=3)
filtered_df = filtered_df[filtered_df['created_time'] < five_months_ago]

In [156]:
filtered_df = filtered_df[~filtered_df['attributes.base_token_price_usd'].between(0.89, 1.11)]
print(f"Количество подходящих строк : {len(filtered_df)}")

Количество подходящих строк : 87


In [157]:
filtered_df['first_part'] = filtered_df['attributes.name'].str.split(' / ').str[0]

In [158]:
bad_name = ['WBTC', 'WETH', "SOL", "BTC", "ETH", "BITCOIN"]
for name in bad_name:
    filtered_df = filtered_df[~filtered_df['first_part'].str.contains(name, case=False, na=False)]
    print(f"Количество подходящих строк : {len(filtered_df)}, {name}")

Количество подходящих строк : 83, WBTC
Количество подходящих строк : 72, WETH
Количество подходящих строк : 71, SOL
Количество подходящих строк : 68, BTC
Количество подходящих строк : 49, ETH
Количество подходящих строк : 47, BITCOIN


In [159]:
names_to_remove = [
                    'DAI / USDC 0.01%', 'USDC / WETH 0.3%'
                    'USDC / WBTC / WETH', 'USDC / USDT 0.01%', 'USDe / DAI',
                   ]

# Фильтруем DataFrame, исключая строки, где attributes.name соответствует элементам из списка
filtered_df = filtered_df[~filtered_df['attributes.name'].isin(names_to_remove)]

In [160]:
df[df['attributes.name'] == 'pufETH / wstETH']

,id,type,attributes.base_token_price_usd,attributes.base_token_price_native_currency,attributes.quote_token_price_usd,attributes.quote_token_price_native_currency,attributes.base_token_price_quote_token,attributes.quote_token_price_base_token,attributes.address,attributes.name,...,attributes.volume_usd.h6,attributes.volume_usd.h24,attributes.reserve_in_usd,relationships.base_token.data.id,relationships.base_token.data.type,relationships.quote_token.data.id,relationships.quote_token.data.type,relationships.dex.data.id,relationships.dex.data.type,created_time
16,0xeeda34a377dd0ca676b9511ee1324974fa8d980d,pool,2682.052404,1.020817,3105.989316,1.182204,0.863487,1.158095,0xeeda34a377dd0ca676b9511ee1324974fa8d980d,pufETH / wstETH,...,4.101054e+06,1.170071e+07,4.658225e+07,eth_0xd9a442856c234a39a81a089c06451ebaa4306a72,token,eth_0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,token,curve,dex,2024-03-03 15:26:16


In [161]:
filtered_df["attributes.name"].unique()

array(['PEPE / WETH', 'UNI / WETH 0.3%', 'LINK / WETH 0.3%',
       'PAXG / WETH', 'Mog / WETH', 'FLOKI / WETH', 'DRAGONX / TITANX 1%',
       'BEAM / WETH', 'FET / WETH 0.3%', 'BANANA / WETH',
       'SHFL / USDC 0.3%', 'SPX / WETH', 'SUPER / WETH',
       'AAVE / WETH 0.3%', 'EIGEN / WETH 0.3%', 'ONDO / WETH 0.3%',
       'TRUMP / WETH', '0x0 / WETH', 'LDO / WETH 0.3%', 'WOJAK / WETH',
       'TITANX / WETH 1%', 'PEPE / WETH 0.3%', 'wTAO / WETH 0.3%',
       'APU / WETH 1%', 'pepecoin / WETH', 'DEAI / WETH 0.3%',
       'APU / WETH', 'ENS / WETH 0.3%', 'AMPL / WETH', 'ENA / WETH 0.3%',
       'NPC / WETH', 'ANDY / WETH', 'PRIME / WETH 0.3%',
       'wQUIL / WETH 1%', 'BOBO / WETH', 'MAGA / WETH', 'HOPPY / WETH',
       'JESUS / WETH', 'JOE / WETH', 'ZYN / WETH', 'WOLF / WETH',
       'SMURFCAT / WETH', 'PENDLE / WETH 0.3%', 'WLD / WETH 1%',
       'GME / WETH', 'FIGHT / WETH', 'Kendu / WETH'], dtype=object)

In [162]:
filtered_df

,id,type,attributes.base_token_price_usd,attributes.base_token_price_native_currency,attributes.quote_token_price_usd,attributes.quote_token_price_native_currency,attributes.base_token_price_quote_token,attributes.quote_token_price_base_token,attributes.address,attributes.name,...,attributes.volume_usd.h24,attributes.reserve_in_usd,relationships.base_token.data.id,relationships.base_token.data.type,relationships.quote_token.data.id,relationships.quote_token.data.type,relationships.dex.data.id,relationships.dex.data.type,created_time,first_part
58,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,pool,1.040550e-05,3.924422e-09,2644.921558,1.000000e+00,3.924422e-09,2.548146e+08,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,PEPE / WETH,...,2.893113e+06,4.219626e+07,eth_0x6982508145454ce325ddbe47a25d4ec3d2311933,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v2,dex,2023-04-14 17:21:11,PEPE
106,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,pool,7.400961e+00,2.796513e-03,2644.961404,1.000000e+00,2.796510e-03,3.575900e+02,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,UNI / WETH 0.3%,...,1.473808e+06,2.565014e+07,eth_0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v3,dex,2021-12-29 13:15:46,UNI
86,0xa6cc3c2531fdaa6ae1a3ca84c2855806728693e8,pool,1.144858e+01,4.321754e-03,2644.961404,1.000000e+00,4.321750e-03,2.313900e+02,0xa6cc3c2531fdaa6ae1a3ca84c2855806728693e8,LINK / WETH 0.3%,...,1.949382e+06,2.215372e+07,eth_0x514910771af9ca656af840dff83e8264ecf986ca,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v3,dex,2021-12-29 12:36:53,LINK
163,0x9c4fe5ffd9a9fc5678cfbd93aa2d4fd684b67c4c,pool,2.750287e+03,1.031430e+00,2658.600653,1.000000e+00,1.031430e+00,9.695276e-01,0x9c4fe5ffd9a9fc5678cfbd93aa2d4fd684b67c4c,PAXG / WETH,...,7.962001e+05,1.904162e+07,eth_0x45804880de22913dafe09f4980848ece6ecbaf78,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v2,dex,2021-10-11 01:42:05,PAXG
41,0xc2eab7d33d3cb97692ecb231a5d0e4a649cb539d,pool,2.107092e-06,7.943121e-10,2644.921558,1.000000e+00,7.943121e-10,1.258951e+09,0xc2eab7d33d3cb97692ecb231a5d0e4a649cb539d,Mog / WETH,...,4.080013e+06,1.812396e+07,eth_0xaaee1a9723aadb7afa2810263653a34ba2c21c7a,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v2,dex,2023-07-20 03:06:47,Mog
56,0xca7c2771d248dcbe09eabe0ce57a62e18da178c0,pool,1.521488e-04,5.735293e-08,2644.921558,1.000000e+00,6.000000e-08,1.743590e+07,0xca7c2771d248dcbe09eabe0ce57a62e18da178c0,FLOKI / WETH,...,3.024297e+06,1.767214e+07,eth_0xcf0c122c6b73ff809c693db761e7baebe62b6a2e,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v2,dex,2022-01-23 00:40:24,FLOKI
90,0x25215d9ba4403b3da77ce50606b54577a71b7895,pool,1.640446e-05,6.265099e-09,0.000001,5.352905e-10,1.170000e+01,8.544006e-02,0x25215d9ba4403b3da77ce50606b54577a71b7895,DRAGONX / TITANX 1%,...,1.775233e+06,1.287814e+07,eth_0x96a5399d07896f757bd4c6ef56461f58db951862,token,eth_0xf19308f923582a6f7c465e5ce7a9dc1bec6665b1,token,uniswap_v3,dex,2024-01-17 21:41:23,DRAGONX
156,0x180efc1349a69390ade25667487a826164c9c6e4,pool,1.731817e-02,6.528317e-06,2644.913246,1.000000e+00,6.530000e-06,1.531790e+05,0x180efc1349a69390ade25667487a826164c9c6e4,BEAM / WETH,...,8.207110e+05,1.106078e+07,eth_0x62d0a8458ed7719fdaf978fe5929c6d342b0bfce,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v2,dex,2023-10-26 10:57:11,BEAM
155,0x744159757cac173a7a3ecf5e97adb10d1a725377,pool,1.431737e+00,5.410299e-04,2646.150607,1.000000e+00,5.410300e-04,1.848330e+03,0x744159757cac173a7a3ecf5e97adb10d1a725377,FET / WETH 0.3%,...,8.234870e+05,1.074928e+07,eth_0xaea46a60368a7bd060eec7df8cba43b7ef41ad85,token,eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,token,uniswap_v3,dex,2022-06-25 23:48:21,FET
83,0x43de4318b6eb91a7cf37975dbb574396a7b5b5c6,pool,5.910959e+01,2.229130e-02,2644.913246,1.000000e+00,2.229130e-02,4.486000e+01,0x43de4318b6eb91a7cf37975dbb574396a7b5b5c6,BANANA / WETH,...,2.11410